In [69]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [70]:
# 데이터 예시 (지역, 가격, 면적, 설명)
data = {'지역': ['강남', '송파', '강북','강남'],
        '가격': [300000, 250000, 200000,290000],
        '면적': [85, 75, 65,80],
        '설명': ['넓고 쾌적한 아파트', '교통이 편리한 지역', '저렴한 아파트','넓은 아파트']}

df = pd.DataFrame(data)

In [71]:
df.head(3)

,지역,가격,면적,설명
0,강남,300000,85,넓고 쾌적한 아파트
1,송파,250000,75,교통이 편리한 지역
2,강북,200000,65,저렴한 아파트


In [72]:
# 1. 범주형 변수 (지역) 벡터화 (One-Hot Encoding)
one_hot = OneHotEncoder()
location_encoded1 = one_hot.fit_transform(df[['지역']]).toarray()
# 레이블링 해도 되지 않을까 |

In [73]:
location_encoded1

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [74]:
# LabelEncoder 사용
label_encoder = LabelEncoder()
location_encoded2 = label_encoder.fit_transform(df['지역'])

In [75]:
location_encoded2   #1차원 배열임
# 뒤에서 합쳐주려면 2차원으로 변경해야함 

location_encoded2 = location_encoded2.reshape(-1,1)


In [76]:
# 2. 수치형 변수 (가격, 면적) 벡터화 (정규화)
scaler = StandardScaler()
numeric_features = scaler.fit_transform(df[['가격', '면적']])

In [77]:
numeric_features

array([[ 1.01600102,  1.18321596],
       [-0.25400025, -0.16903085],
       [-1.52400152, -1.52127766],
       [ 0.76200076,  0.50709255]])

In [78]:
# 3. 텍스트 변수 (설명) 벡터화 (TF-IDF)
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['설명']).toarray()

In [79]:
tfidf_matrix.shape

(4, 8)

In [80]:
# 4. 모든 변수를 결합하여 하나의 벡터로
import numpy as np
final_matrix = np.hstack((location_encoded2, numeric_features, tfidf_matrix))
#location_encoded(3개)/numeric_features(2개)/tfidf_matrix(7개) 총 12

print(final_matrix)
print(final_matrix.shape)

[[ 0.          1.01600102  1.18321596  0.          0.64450299  0.
   0.41137791  0.          0.          0.64450299  0.        ]
 [ 2.         -0.25400025 -0.16903085  0.57735027  0.          0.
   0.          0.          0.57735027  0.          0.57735027]
 [ 1.         -1.52400152 -1.52127766  0.          0.          0.
   0.53802897  0.84292635  0.          0.          0.        ]
 [ 0.          0.76200076  0.50709255  0.          0.          0.84292635
   0.53802897  0.          0.          0.          0.        ]]
(4, 11)


In [81]:
final_matrix

array([[ 0.        ,  1.01600102,  1.18321596,  0.        ,  0.64450299,
         0.        ,  0.41137791,  0.        ,  0.        ,  0.64450299,
         0.        ],
       [ 2.        , -0.25400025, -0.16903085,  0.57735027,  0.        ,
         0.        ,  0.        ,  0.        ,  0.57735027,  0.        ,
         0.57735027],
       [ 1.        , -1.52400152, -1.52127766,  0.        ,  0.        ,
         0.        ,  0.53802897,  0.84292635,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.76200076,  0.50709255,  0.        ,  0.        ,
         0.84292635,  0.53802897,  0.        ,  0.        ,  0.        ,
         0.        ]])

In [82]:
from sklearn.metrics.pairwise import cosine_similarity

In [83]:
# 매물 간 유사도 계산
similarity_matrix = cosine_similarity(final_matrix)

In [84]:
# 유사도 행렬 출력
print(similarity_matrix)

[[ 1.         -0.10955851 -0.65518487  0.63528181]
 [-0.10955851  1.          0.45480864 -0.09127979]
 [-0.65518487  0.45480864  1.         -0.4705142 ]
 [ 0.63528181 -0.09127979 -0.4705142   1.        ]]


In [85]:
# 0번째 매물과 가장 유사한 매물 3개 추천 (자기 자신 제외)
similarity_scores = list(enumerate(similarity_matrix[0]))
# 지금은 0번째 매물이라고 했지만 
# 매물 아이디값? 을 직접 넣어주도록 함수의매개변수로 지정 
similarity_scores

[(0, 1.0),
 (1, -0.10955850876091178),
 (2, -0.6551848680889083),
 (3, 0.6352818146321013)]

In [86]:
# 유사도 점수를 기준으로 내림차순 정렬
sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

In [87]:
# 상위 3개의 유사한 매물 출력 (자기 자신 제외)
top_similar_properties = sorted_scores[1:4]
top_similar_properties
print("\n0번째 매물과 유사한 상위 3개 매물:\n", top_similar_properties)


0번째 매물과 유사한 상위 3개 매물:
 [(3, 0.6352818146321013), (1, -0.10955850876091178), (2, -0.6551848680889083)]
